### Make a Multivariate Regress with Loess

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import linalg

$ Y = X \beta $

We solve a problem like:

$$ \large W_i \cdot Y = Wi \cdot X \cdot \beta $$

We give it $W_i$ diagonal matrices, and learn $\beta$ along the way, solving this equation to get $Y$ predictions

Then solve equation $A \cdot \beta = b$

Then b is a linear combination of the columns of A. We can see this as an orthogonal projection of b onto the vector subspace spanned by the columns of A.

In [5]:
#Define some kernals
def tricubic(dx):
    if len(dx.shape) == 1:
        dx = dx.reshape(-1,1)
    d = np.sqrt(np.sum(dx**2, axis=1))
    return np.where(d>1,0,70/81*(1-d**3)**3)

In [6]:
def lw_reg(X, y, xnew, kern, tau, intercept):
    # tau is called bandwidth K((x-x[i])/(2*tau))
    n = len(X)
    yest = np.zeros(n)

    if len(y.shape)==1:
      y = y.reshape(-1,1)

    if len(X.shape)==1:
      X = X.reshape(-1,1)
    
    if intercept:
      X1 = np.column_stack([np.ones((len(X),1)),X])
    else:
      X1 = X

    w = np.array([kern((X - X[i])/(2*tau)) for i in range(n)]) 

    #Looping through all X-points
    for i in range(n):          
        W = np.diag(w[:,i])
        b = np.transpose(X1).dot(W).dot(y)
        A = np.transpose(X1).dot(W).dot(X1)
        #A = A + 0.001*np.eye(X1.shape[1]) # if we want L2 regularization
        #theta = linalg.solve(A, b) # A*theta = b
        theta, res, rnk, s = lstsq(A, b)
        yest[i] = np.dot(X1[i],theta)
    if X.shape[1]==1:
      f = interp1d(X.flatten(),yest,fill_value='extrapolate')
    else:
      f = LinearNDInterpolator(X, yest)
    output = f(xnew)
    if sum(np.isnan(output))>0:
      g = NearestNDInterpolator(X,y.ravel()) 
      output[np.isnan(output)] = g(X[np.isnan(output)])
    return output

In [10]:
data= pd.read_csv("Data/cars.csv")
X = data[['CYL', "ENG", "WGT"]].values
Y = data['MPG']
xnew = [4,265,2873]
lw_reg(X,Y,xnew,tricubic,20, True)

AttributeError: 'Series' object has no attribute 'reshape'